In [6]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import time

In [7]:
import requests
import bs4
import json

# Scraping an Online Bookstore




http://books.toscrape.com/. This website is meant for toying with scraping.


- function `extract_book_links` that takes in the content of a book-listing page (a string of html), and returns a list of urls of book-detail pages that satisfy the requirements on "*at least* a four-star rating, and prices are *under* £50". 

- function `get_product_info` that takes in the content of a book-detail page (a string of html), a variable `categories` that is a list of book categories you want. If this input book is in the categories you want, returns a dictionary corresponding to a row in the dataframe in the image above (where the keys are the column names and the values are the row values); else, skip this book since this is not the book you want (ie. return None).

- function `scrape_books` of a single variable `k` that scrapes the first `k` pages of the bookstore (as determined by starting at the url above and clicking on the 'next' button),a variable `categories` that is a list of book categories you want, and returns a dataframe of books as the picture above. (Note: make sure the books returned satisfy the requirements set in part 1 about rating and price).


In [13]:
fp = os.path.join('data', 'products.html')
text = open(fp, encoding='utf-8').read()

In [14]:

soup = bs4.BeautifulSoup(text)
#soup



In [15]:
articles = soup.findAll('article')
#articles[0].find('a').get('href')

In [1025]:
def extract_book_links(text):
    """
    :Example:
    >>> fp = os.path.join('data', 'products.html')
    >>> out = extract_book_links(open(fp, encoding='utf-8').read())
    >>> url = 'scarlet-the-lunar-chronicles-2_218/index.html'
    >>> out[1] == url
    True
    """
    
    soup = bs4.BeautifulSoup(text, 'html.parser')
    articles = soup.find_all('article', attrs={'class': 'product_pod'})
    urls = []
    for article in articles:
        price = article.find('p', attrs='price_color').text
        price = float(price.split('£')[1])
        four_rating = article.find('p', attrs='star-rating Four')
        five_rating = article.find('p', attrs='star-rating Five')

        if (price < 50) and ((four_rating != None) | (five_rating != None)):
            url = article.find('a').get('href')
            urls.append(url)

    return urls

In [1026]:
fp = os.path.join('data', 'products.html')
out = lab.extract_book_links(open(fp, encoding='utf-8').read())
url = 'scarlet-the-lunar-chronicles-2_218/index.html'
out[1] == url

True

function get_product_info that takes in the content of a book-detail page (a string of html), a variable categories that is a list of book categories you want. If this input book is in the categories you want, returns a dictionary corresponding to a row in the dataframe in the image above (where the keys are the column names and the values are the row values); else, skip this book since this is not the book you want (ie. return None).

In [34]:
def get_product_info(text, categories):
    """
    :Example:
    >>> fp = os.path.join('data', 'Frankenstein.html')
    >>> out = get_product_info(open(fp, encoding='utf-8').read(), ['Default'])
    >>> isinstance(out, dict)
    True
    >>> 'Category' in out.keys()
    True
    >>> out['Rating']
    'Two'
    """
    soup = bs4.BeautifulSoup(text, 'html.parser')
    cate = soup.find_all('a')[3].text
    if cate in categories:
        des = soup.find_all('meta')[2].get('content').strip()
        tds = soup.find_all('td')
        tdv = [td.text for td in tds]
        ttl = soup.find('li', attrs = {'class':'active'}).text
        rating = soup.find_all('p')[2].get('class')[-1]
        dic = dict({
                'Availability': tdv[5],
                'Category': cate,
                'Description': des,
                'Number of reviews': tdv[-1],
                'Price (excl. tax)': tdv[2],
                'Price (incl. tax)': tdv[3],
                'Product Type': tdv[1],
                'Rating': rating,
                'Tax': tdv[4],
                'Title': ttl,
                'UPC': tdv[0]
            })
        return dic

function scrape_books of a single variable k that scrapes the first k pages of the bookstore (as determined by starting at the url above and clicking on the 'next' button),a variable categories that is a list of book categories you want, and returns a dataframe of books as the picture above. (Note: make sure the books returned satisfy the requirements set in part 1 about rating and price).

In [1016]:
def scrape_books(k, categories):
    """
    :param k: number of book-listing pages to scrape.
    :returns: a dataframe of information on (certain) books
    on the k pages (as described in the question).

    :Example:
    >>> out = scrape_books(1, ['Mystery'])
    >>> out.shape
    (1, 11)
    >>> out['Rating'][0] == 'Four'
    True
    >>> out['Title'][0] == 'Sharp Objects'
    True
    """
    prefix = 'http://books.toscrape.com/catalogue/page-'
    pages = [prefix + str(i) +'.html' for i in range(1,k+1)]
    
    df_lst = []
    for page in pages:
        resp = requests.get(page)
        text = resp.text
        urls = lab.extract_book_links(text)
        for url in urls:
            path = 'http://books.toscrape.com/catalogue/' + url
            req = requests.get(path)
            txt = req.text
            dic = lab.get_product_info(txt, categories)
            if dic != None:
                df = pd.DataFrame([dic])
                df_lst.append(df)
                

    res = pd.concat(df_lst, ignore_index=True)
    
    return res

In [37]:
out = lab.scrape_books(6, ['Mystery', 'Default', 'Business'])
out

,Availability,Category,Description,Number of reviews,Price (excl. tax),Price (incl. tax),Product Type,Rating,Tax,Title,UPC
0,In stock (20 available),Mystery,"WICKED above her hipbone, GIRL across her hear...",0,Â£47.82,Â£47.82,Books,Four,Â£0.00,Sharp Objects,e00eb4fd7b871a48
1,In stock (19 available),Business,Drawing on his extensive experience evaluating...,0,Â£33.34,Â£33.34,Books,Four,Â£0.00,The Dirty Little Secrets of Getting Your Dream...,2597b5a345f45e1b
2,In stock (19 available),Default,For readers of Laura Hillenbrand's Seabiscuit ...,0,Â£22.60,Â£22.60,Books,Four,Â£0.00,The Boys in the Boat: Nine Americans and Their...,e10e1e165dc8be4a
3,In stock (16 available),Default,"Slay Procrastination, Distraction, and Overwhe...",0,Â£20.59,Â£20.59,Books,Five,Â£0.00,The Inefficiency Assassin: Time Management Tac...,8c9e6bf2467d740d


In [47]:
import datetime

In [61]:
dd = datetime.datetime.now()
lab.scrape_books(50, ['Mystery'])
dur = datetime.datetime.now() - dd

In [62]:
dur

datetime.timedelta(seconds=266, microseconds=34542)

# API Requests

function `stock_history` which takes in the stock code (`ticker`) as a string, `year` and `month` as integers, and return a dataframe which has the price history for that stock in that month (include all columns).

function `stock_stats` that takes in the output dataframe from `stock_history` and output the stock price change as a percentage and a rough total transaction volume **in billion dollars** for that month. Assume that on average, shares are traded at the midpoint price of high and low for that day. Return these two values as a tuple in a readable format: reserve 2 decimal points for both values and add a plus or minus sign at the front of the percent change. 
$$ \text{Total Transaction Volume (in dollars)} = \text{Volume (number of shares traded)} \times \text{Price} $$




In [86]:
stock_endpoint = 'https://financialmodelingprep.com/api/v3/historical-price-full/{}?apikey=4727f0e603c0d487db9a927d1f4f8645'.format('BYND')
stock_endpoint

'https://financialmodelingprep.com/api/v3/historical-price-full/BYND?apikey=4727f0e603c0d487db9a927d1f4f8645'

In [89]:
content = requests.get(stock_endpoint).json()
content

{'symbol': 'BYND',
 'historical': [{'date': '2021-11-08',
   'open': 99.9,
   'high': 99.99,
   'low': 96.25,
   'close': 98.28,
   'adjClose': 98.28,
   'volume': 2325920.0,
   'unadjustedVolume': 2325920.0,
   'change': -1.62,
   'changePercent': -1.622,
   'vwap': 98.17333,
   'label': 'November 08, 21',
   'changeOverTime': -0.01622},
  {'date': '2021-11-05',
   'open': 102.0,
   'high': 102.6,
   'low': 99.12,
   'close': 99.77,
   'adjClose': 99.77,
   'volume': 2242820.0,
   'unadjustedVolume': 2242820.0,
   'change': -2.23,
   'changePercent': -2.186,
   'vwap': 100.49667,
   'label': 'November 05, 21',
   'changeOverTime': -0.02186},
  {'date': '2021-11-04',
   'open': 104.51,
   'high': 107.2,
   'low': 102.5,
   'close': 102.52,
   'adjClose': 102.52,
   'volume': 2352795.0,
   'unadjustedVolume': 2352795.0,
   'change': -1.99,
   'changePercent': -1.904,
   'vwap': 104.07333,
   'label': 'November 04, 21',
   'changeOverTime': -0.01904},
  {'date': '2021-11-03',
   'open': 

In [1104]:
content['historical'][0]

{'date': '2021-05-07',
 'open': 107.73,
 'high': 116.88,
 'low': 106.91,
 'close': 110.73,
 'adjClose': 110.73,
 'volume': 9340258.0,
 'unadjustedVolume': 9340258.0,
 'change': 3.0,
 'changePercent': 2.785,
 'vwap': 111.50667,
 'label': 'May 07, 21',
 'changeOverTime': 0.02785}

In [1110]:
res = pd.DataFrame([x for x in content['historical'] if x['date'][:7] == '2019-06'])
res.head()

,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime
0,2019-06-28,165.300003,168.800003,159.550003,160.679993,160.679993,7315300.0,7315300.0,-4.62001,-2.795,163.01000,"June 28, 19",-0.02795
1,2019-06-27,157.309998,164.789993,155.449997,162.910004,162.910004,5731400.0,5731400.0,5.60001,3.560,161.05000,"June 27, 19",0.03560
2,2019-06-26,160.100006,162.250000,153.020004,160.479996,160.479996,6378600.0,6378600.0,0.37999,0.237,158.58333,"June 26, 19",0.00237
3,2019-06-25,138.500000,150.690002,138.343002,150.600006,150.600006,6632500.0,6632500.0,12.10001,8.736,146.54434,"June 25, 19",0.08736
4,2019-06-24,151.880005,152.699997,138.000000,140.990005,140.990005,6538500.0,6538500.0,-10.89000,-7.170,143.89667,"June 24, 19",-0.07170


In [1114]:
def stock_history(ticker, year, month):
    """
    Given a stock code and month, return the stock price details for that month
    as a dataframe

    >>> history = stock_history('BYND', 2019, 6)
    >>> history.shape == (20, 13)
    True
    >>> history.label.iloc[-1]
    'June 03, 19'
    """
    stock_endpoint = 'https://financialmodelingprep.com/api/v3/historical-price-full/{}?apikey=4727f0e603c0d487db9a927d1f4f8645'.format(ticker)
    content = requests.get(stock_endpoint).json()
    res = pd.DataFrame([x for x in content['historical'] if x['date'][:7] == '2019-06'])
    return res

In [91]:
ticker = 'BYND'

In [95]:
stock_endpoint = 'https://financialmodelingprep.com/api/v3/historical-price-full/{}?apikey=4727f0e603c0d487db9a927d1f4f8645'.format(ticker)
content = requests.get(stock_endpoint).json()
pd.DataFrame([x for x in content['historical']])

,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime
0,2021-11-08,99.900000,99.990000,96.250000,98.280000,98.280000,2325920.0,2325920.0,-1.6200,-1.622,98.17333,"November 08, 21",-0.01622
1,2021-11-05,102.000000,102.600000,99.120000,99.770000,99.770000,2242820.0,2242820.0,-2.2300,-2.186,100.49667,"November 05, 21",-0.02186
2,2021-11-04,104.510000,107.200000,102.500000,102.520000,102.520000,2352795.0,2352795.0,-1.9900,-1.904,104.07333,"November 04, 21",-0.01904
3,2021-11-03,99.700000,108.040000,99.660000,104.510000,104.510000,5453031.0,5453031.0,4.8100,4.824,104.07000,"November 03, 21",0.04824
4,2021-11-02,99.844200,100.170000,97.270000,99.190000,99.190000,1877917.0,1877917.0,-0.6542,-0.655,98.87667,"November 02, 21",-0.00655
...,...,...,...,...,...,...,...,...,...,...,...,...,...
632,2019-05-08,83.610001,85.379997,70.785004,72.250000,72.250000,14711800.0,14711800.0,-11.3600,-13.587,76.13833,"May 08, 19",-0.13587
633,2019-05-07,77.139999,85.449997,75.000000,79.169998,79.169998,16532100.0,16532100.0,2.0300,2.632,79.87333,"May 07, 19",0.02632
634,2019-05-06,62.730000,74.839996,62.500000,74.790001,74.790001,8746200.0,8746200.0,12.0600,19.225,70.71000,"May 06, 19",0.19225
635,2019-05-03,72.000000,74.000000,65.660004,66.790001,66.790001,13139400.0,13139400.0,-5.2100,-7.236,68.81667,"May 03, 19",-0.07236


In [96]:
history = lab.stock_history('BYND', 2019, 6)
history.shape == (20, 13)

True

In [1211]:
def stock_stats(history):
    """
    Given a stock's trade history, return the percent change and transactions
    in billion dollars.

    >>> history = stock_history('BYND', 2019, 6)
    >>> stats = stock_stats(history)
    >>> len(stats[0]), len(stats[1])
    (7, 6)
    >>> float(stats[0][1:-1]) > 30
    True
    >>> float(stats[1][:-1]) > 1
    True
    """
    fday = history['date'].values[-1]
    fopen = history.loc[history['date'] == fday, 'open'].values[0]
    lday = history['date'].values[0]
    lclose = history.loc[history['date'] == lday, 'close'].values[0]
    pct_change = (lclose - fopen) / fopen * 100
    if pct_change > 0:
        pct_change = "+{:0.2f}%".format(pct_change)
    else:
        pct_change = "-{:0.2f}%".format(pct_change)
        
    vol = sum((history['high'] + history['low']) / 2 * history['volume']) * 10**(-9)
    vol = "{:0.2f}B".format(vol)
    
    return (pct_change, vol)

# Comment Threads


As a hacker, you get your daily dose of tech news on [Hacker News](https://news.ycombinator.com/). The problem now is that you don't have internet access on your phone in your morning commute to work, so you want to save the interesting stories' comments thread beforehand in a flat file source like csv. You find their API documentation ( https://github.com/HackerNews/API) and implement the following task:

function `get_comments` that takes `storyid` as a parameter and returns a dataframe of all the comments below the news story. You can ignore 'dead' comments (you will know it when you see it). **Make sure the order of the comments in your dataframe is from top to bottom just as you see on the website**. You are allowed to use loops in this function. Addtional requirement: write at least one helper method

You only want these information for the comments:
1. `id`: the unique ids
2. `by`: the author of the comment
3. `parent`: who (also in unique ids) they are replying to
4. `text`: the actual comment
5. `time`: when the comment is created (in `pd.datetime` format)


In [120]:
# depth first search

def dfs(content):
    if 'kids' not in content.keys():
        return 
    for id in content['kids']:
        endpoint = "https://hacker-news.firebaseio.com/v0/item/{}.json".format(id)
        con = requests.get(endpoint).json()
        lst.append(con)
        dfs(con)
    return lst

In [121]:
res = dfs(content)
res

[{'by': 'valyala',
  'id': 18380397,
  'parent': 18344932,
  'text': 'TimescaleDB is great for storing time series comparing to vanilla ProstgreSQL!<p>Unfortunately it sometimes looses in storage cost-effectiveness comparing to competing TSDBs - <a href="https:&#x2F;&#x2F;medium.com&#x2F;@valyala&#x2F;when-size-matters-benchmarking-victoriametrics-vs-timescale-and-influxdb-6035811952d4" rel="nofollow">https:&#x2F;&#x2F;medium.com&#x2F;@valyala&#x2F;when-size-matters-benchmarking-v...</a>',
  'time': 1541400799,
  'type': 'comment'},
 {'by': 'msiggy',
  'id': 18346406,
  'parent': 18344932,
  'text': 'I&#x27;m excited to give this database a try if I can find some free time.',
  'time': 1540999222,
  'type': 'comment'},
 {'by': 'sman393',
  'id': 18348601,
  'kids': [18348631],
  'parent': 18344932,
  'text': 'Can this be used side by side on normal Postgres cluster? As in could I have one DB for app data, and one for metrics data? Considering switching from MySQL (ndb cluster) to runni

In [122]:
df = pd.DataFrame(res)
df

,by,id,parent,text,time,type,kids,dead
0,valyala,18380397,18344932,TimescaleDB is great for storing time series c...,1541400799,comment,NaN,NaN
1,msiggy,18346406,18344932,I&#x27;m excited to give this database a try i...,1540999222,comment,NaN,NaN
2,sman393,18348601,18344932,Can this be used side by side on normal Postgr...,1541014179,comment,[18348631],NaN
3,RobAtticus,18348631,18348601,"Yep, absolutely. Regular PostgreSQL tables coe...",1541014492,comment,[18348984],NaN
4,sman393,18348984,18348631,Good to hear! how does the current TimescaleDB...,1541017426,comment,[18349540],NaN
5,RobAtticus,18349540,18348984,Not sure I follow exactly what you&#x27;re ask...,1541022440,comment,[18350673],NaN
6,sman393,18350673,18349540,Alright thanks! I thought I read that Timescal...,1541034719,comment,[18351061],NaN
7,RobAtticus,18351061,18350673,It does not support sharding writes across mul...,1541039703,comment,NaN,NaN
8,zip1234,18346750,18344932,How fast is it when it has a TB of data? I rea...,1541001103,comment,"[18347260, 18347555]",NaN
9,nevi-me,18347260,18346750,I spent about 8 months writing data to TSDB. I...,1541004454,comment,NaN,NaN


In [123]:
df = df[df['dead'] != True]
df
result = df.copy()

In [124]:
result['time'] = df['time'].apply(pd.Timestamp, unit = 's')
result


,by,id,parent,text,time,type,kids,dead
0,valyala,18380397,18344932,TimescaleDB is great for storing time series c...,2018-11-05 06:53:19,comment,NaN,NaN
1,msiggy,18346406,18344932,I&#x27;m excited to give this database a try i...,2018-10-31 15:20:22,comment,NaN,NaN
2,sman393,18348601,18344932,Can this be used side by side on normal Postgr...,2018-10-31 19:29:39,comment,[18348631],NaN
3,RobAtticus,18348631,18348601,"Yep, absolutely. Regular PostgreSQL tables coe...",2018-10-31 19:34:52,comment,[18348984],NaN
4,sman393,18348984,18348631,Good to hear! how does the current TimescaleDB...,2018-10-31 20:23:46,comment,[18349540],NaN
5,RobAtticus,18349540,18348984,Not sure I follow exactly what you&#x27;re ask...,2018-10-31 21:47:20,comment,[18350673],NaN
6,sman393,18350673,18349540,Alright thanks! I thought I read that Timescal...,2018-11-01 01:11:59,comment,[18351061],NaN
7,RobAtticus,18351061,18350673,It does not support sharding writes across mul...,2018-11-01 02:35:03,comment,NaN,NaN
8,zip1234,18346750,18344932,How fast is it when it has a TB of data? I rea...,2018-10-31 15:51:43,comment,"[18347260, 18347555]",NaN
9,nevi-me,18347260,18346750,I spent about 8 months writing data to TSDB. I...,2018-10-31 16:47:34,comment,NaN,NaN


In [20]:
def get_comments(storyid):
    """
    Returns a dataframe of all the comments below a news story
    >>> out = get_comments(18344932)
    >>> out.shape
    (18, 5)
    >>> out.loc[5, 'by']
    'RobAtticus'
    >>> out.loc[5, 'time'].day
    31
    """
    news_endpoint = "https://hacker-news.firebaseio.com/v0/item/{}.json".format(storyid)
    content = requests.get(news_endpoint).json()
    lst = []
    
    def dfs(content):
        if 'kids' not in content.keys():
            return 
        for id in content['kids']:
            endpoint = "https://hacker-news.firebaseio.com/v0/item/{}.json".format(id)
            con = requests.get(endpoint).json()
            lst.append(con)
            dfs(con)
        return lst
    
    res = dfs(content)
    df = pd.DataFrame(res)
    df = df[df['dead'] != True]
    df = df.drop(['type', 'dead', 'kids'], axis = 1)   
    
    result = df.copy()
    result['time'] = df['time'].apply(pd.Timestamp, unit = 's')
    
    return result

In [41]:
out

,by,id,parent,text,time
0,valyala,18380397,18344932,TimescaleDB is great for storing time series c...,2018-11-05 06:53:19
1,msiggy,18346406,18344932,I&#x27;m excited to give this database a try i...,2018-10-31 15:20:22
2,sman393,18348601,18344932,Can this be used side by side on normal Postgr...,2018-10-31 19:29:39
3,RobAtticus,18348631,18348601,"Yep, absolutely. Regular PostgreSQL tables coe...",2018-10-31 19:34:52
4,sman393,18348984,18348631,Good to hear! how does the current TimescaleDB...,2018-10-31 20:23:46
5,RobAtticus,18349540,18348984,Not sure I follow exactly what you&#x27;re ask...,2018-10-31 21:47:20
6,sman393,18350673,18349540,Alright thanks! I thought I read that Timescal...,2018-11-01 01:11:59
7,RobAtticus,18351061,18350673,It does not support sharding writes across mul...,2018-11-01 02:35:03
8,zip1234,18346750,18344932,How fast is it when it has a TB of data? I rea...,2018-10-31 15:51:43
9,nevi-me,18347260,18346750,I spent about 8 months writing data to TSDB. I...,2018-10-31 16:47:34
